<a href="https://colab.research.google.com/github/jedhouas/Alzheimer/blob/master/Deep_Image_Harmonization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/wasidennis/DeepHarmonization.git

Cloning into 'DeepHarmonization'...
remote: Enumerating objects: 193, done.
remote: Total 193 (delta 0), reused 0 (delta 0), pack-reused 193
Receiving objects: 100% (193/193), 40.79 MiB | 26.98 MiB/s, done.
Resolving deltas: 100% (54/54), done.


In [2]:
!apt install caffe-cpu

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  caffe-tools-cpu cython3 fonts-lyx javascript-common libblosc1 libcaffe-cpu1
  libgflags2.2 libgoogle-glog0v5 libjs-jquery-ui libleveldb1v5 liblmdb0
  python-matplotlib-data python-tables-data python3-bs4 python3-caffe-cpu
  python3-chardet python3-cycler python3-dateutil python3-decorator
  python3-gflags python3-h5py python3-html5lib python3-ipython
  python3-ipython-genutils python3-leveldb python3-lxml python3-matplotlib
  python3-networkx python3-nose python3-numexpr python3-olefile python3-pandas
  python3-pandas-lib python3-pexpect python3-pickleshare python3-pil
  python3-pkg-resources python3-prompt-toolkit python3-protobuf
  python3-ptyprocess python3-pygments python3-pyparsing python3-pywt
  pyt

In [3]:
!python /content/DeepHarmonization/demo.py

  File "/content/DeepHarmonization/demo.py", line 61
    neg_idx = out < 0.0
                      ^
TabError: inconsistent use of tabs and spaces in indentation


In [0]:
import caffe
import numpy as np
from PIL import Image
import os

# result folder
folder_name = 'result/'
if os.path.isdir(folder_name):
  pass
else:
  os.makedirs(folder_name)

# load test image list
filename = 'data/list.txt'
#with open(filename, 'r') as f:
#  path_src = [line.rstrip() for line in f.readlines()]

# set up caffe
#caffe.set_device(0)
#caffe.set_mode_gpu()
caffe.set_mode_cpu()
# load net
net = caffe.Net('/content/DeepHarmonization/model/deploy_512.prototxt', '/content/DeepHarmonization/model/harmonize_iter_200000.caffemodel', caffe.TEST)
size = np.array([512,512])
idx = 1
path_ = "Test1.jpeg"
im_ori = Image.open('/content/DeepHarmonization/data/image/Test1.jpeg')
im = im_ori.resize(size, Image.BICUBIC)
im = np.array(im, dtype=np.float32)
if im.shape[2] == 4:
	im = im[:,:,0:3]
im = im[:,:,::-1]
im -= np.array((104.00699, 116.66877, 122.67892))
im = im.transpose((2,0,1))
mask = Image.open('/content/DeepHarmonization/data/mask/Test1.jpeg')
mask = mask.resize(size, Image.BICUBIC)
mask = np.array(mask, dtype=np.float32)
if len(mask.shape) == 3:
	mask = mask[:,:,0]
mask -= 128.0
mask = mask[np.newaxis, ...]
# shape for input (data blob is N x C x H x W), set data
net.blobs['data'].reshape(1, *im.shape)
net.blobs['data'].data[...] = im
net.blobs['mask'].reshape(1, *mask.shape)
net.blobs['mask'].data[...] = mask
# run net for prediction
net.forward()
out = net.blobs['output-h'].data[0]
out = out.transpose((1,2,0))
out += np.array((104.00699, 116.66877, 122.67892))
out = out[:,:,::-1]
neg_idx = out < 0.0
out[neg_idx] = 0.0
pos_idx = out > 255.0
out[pos_idx] = 255.0
# save result
result = out.astype(np.uint8)
result = Image.fromarray(result)
im = im_ori.resize(size, Image.BICUBIC);
im = np.array(im, dtype=np.uint8)
if im.shape[2] == 4:
	im = im[:,:,0:3]
end = path_.find('.')
result_all = np.concatenate((im, result), axis = 1)
result_all = Image.fromarray(result_all)
result_all.save(folder_name + path_[0:end] + '.png')

# Nouvelle section

In [5]:
!wget http://vllab.ucmerced.edu/ytsai/CVPR17/harmonize_iter_200000.caffemodel

--2020-01-08 22:42:57--  http://vllab.ucmerced.edu/ytsai/CVPR17/harmonize_iter_200000.caffemodel
Resolving vllab.ucmerced.edu (vllab.ucmerced.edu)... 169.236.184.69
Connecting to vllab.ucmerced.edu (vllab.ucmerced.edu)|169.236.184.69|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 167123847 (159M) [text/plain]
Saving to: ‘harmonize_iter_200000.caffemodel’

harmonize_iter_2000 100%[===================>] 159.38M  18.4MB/s    in 11s     

2020-01-08 22:43:08 (14.5 MB/s) - ‘harmonize_iter_200000.caffemodel’ saved [167123847/167123847]

